[Diffrax - Getting started](https://docs.kidger.site/diffrax/usage/getting-started/)

# Controlled differential equations (CDEs)
このライブラリではODEもSDEも実際にはまったく同じ方法で解かれる。
そのおかげで、比較的小さなライブラリに多くの機能を詰め込むことができている。

例として、次のようなCDEの解を考える。

$$y(0) = 1,  \mathrm{d}y = -y(t)\mathrm{d}x(t)$$ 

区間は $t \in [0, 3]$, 制御信号は $x(t) = t^2$ とする。

In [4]:
from diffrax import AbstractPath, ControlTerm, diffeqsolve, Dopri5
import numpy as np

[3.]
[0.00012341]


## ベクトル場
まずはベクトル場を定義する。今回は $-y$ である.

In [5]:
vector_field = lambda t, y, args: -y # ベクトル場. Neural CDEでは NN が組み込まれる.

## [制御項](https://docs.kidger.site/diffrax/api/path/#diffrax.AbstractPath)
次に制御信号を定義する.
離散的な入力信号を補間して連続的なパスにする際は、 [Interpolations](https://docs.kidger.site/diffrax/api/interpolation/) を用いることになる。

In [ ]:
class QuadraticPath(AbstractPath):
    '''
        制御信号(path; パス)を定義.
        制御信号が入力時系列に依存する場合, Neural CDE の例を参照すると良い.
    '''
    @property
    def t0(self):
        # 開始時刻
        return 0

    @property
    def t1(self):
        # 終端時刻
        return 3

    def evaluate(self, t0, t1=None, left=True):
        del left
        if t1 is not None:
            return self.evaluate(t1) - self.evaluate(t0)
        return t0 ** 2
        
control = QuadraticPath()

## [項](https://docs.kidger.site/diffrax/api/terms/)
Diffrax ではいろんな種類の微分方程式を統一的に扱うために、 `Term`　をいう機能を持っている。
例えば、今回の微分方程式 $$\mathrm{d}y = -y(t)\mathrm{d}x(t)$$ は、ベクトル場 $-y(t)$ という行列と、制御項 $\mathrm{x}(t)$ というベクトルの行列-ベクトル積である。
他の微分方程式もベクトル場と制御項の積の組み合わせで構成されることから、 Diffrax では要素ごとに関数・モデルを定義し、 `Term` によって微分方程式を構築している。

今回は CDEs なので `ControlTerm` を使用する。
`to_ode()` は制御信号が微分可能なときに使用するものである。
制御信号 $x(t)$ が微分可能な時、 $$\mathrm{d}y = f(t, y(t))\mathrm{d}x(t) = f(t, y(t)) \frac{\mathrm{d}x}{\mathrm{d}t}\mathrm{d}t$$ のようにして ODE としてみなすことができる。
今回の制御信号 $x(t) = t^2$ は微分可能なため、 `Control Term` --> `ODETerm` への変換を行なっている。

In [ ]:
term = ControlTerm(vector_field, control).to_ode() # 微分方程式に含まれるこうの定義（ベクトル場と制御項）

## [ソルバー（微分方程式の数値解法）](https://docs.kidger.site/diffrax/api/solvers/ode_solvers/)
ルンゲ=クッタ法の次数や、ステップサイズを固定にするか適応的にするかなどで解法が異なる。
ルンゲ=クッタ法の次数は近似の精度に影響があり、次数が高い解法を使うほど、より精密な矜持となる。

In [ ]:
solver = Dopri5() # ソルバー

## [解](diffeqsolve)
解の計算は `diffeqsolve` を用いる。
この関数はODE、SDE、CDEを問わず、あらゆる種類の初期値問題を解くための主な出発地点となる。
微分方程式は `t0` から `t1` まで積分されます。

**引数について**
- 主な引数
    - `Term`： 微分方程式の項。ベクトル場を指定。(非定常微分方程式(SDEs, CDEs)の場合、これはブラウン運動や制御も指定)
    - `solver`： 微分方程式の解法。
    - `t0`: 積分区間の開始点。
    - `t1`: 積分区間の終了点。
    - `dt0`： 最初のステップに使用するステップサイズ。固定ステップサイズを使用する場合、これは他のすべてのステップのステップサイズにもなります。(最後のステップを除き、若干小さくしてt1にクリップすることも可能)Noneに設定すると、最初のステップサイズが自動的に決定されます。
    - `y0`: 初期値。
    - `args`： ベクトル場に渡す追加の引数。
    - `saveat`: 微分方程式の解を保存する時間。デフォルトは最後の時間 `t1` だけ。[`diffrax.SaveAt`](https://docs.kidger.site/diffrax/api/saveat/#diffrax.SaveAt)
    - `stepize_controller`: 積分の進行に応じてステップサイズを変更する方法。ステップサイズコントローラーのリストを参照。デフォルトは固定された一定のステップサイズを使用する。[`stepsize_controller`](https://docs.kidger.site/diffrax/api/stepsize_controller/)
- その他の引数
    - `adjoint`: diffeqsolveを微分する方法。デフォルトは `discretise-then-optimise` で、ほとんどの問題では通常これが最適。[`Adjoints`](https://docs.kidger.site/diffrax/api/adjoints/)
    - `discrete_terminating_event`： 解を早期に終了させる離散イベント。[`Events`](https://docs.kidger.site/diffrax/api/events/)
    - `max_steps`： `saveat=SaveAt(steps=True)` や `saveat=SaveAt(dense=True)` とは互換性がないが、任意のステップ数を指定するために `None` を設定することも可能。
    - `throw`： `True` の場合、統合に失敗するとエラーが発生する。エラーはCPUでのみ確実に発生することに注意。 `False` の場合、返される解オブジェクトは、何らかの失敗が起こったかどうかを示す結果フィールドを持つことになる。

In [6]:
sol = diffeqsolve(term, solver, t0=0, t1=3, dt0=0.05, y0=1) # 解

In [7]:
print(sol.ts)  # DeviceArray([3.])
print(sol.ys)  # DeviceArray([0.00012341])

[3.]
[0.00012341]
